# WikidWiki Class Description

## Implemented Attributes
- rootfolder
- datafolder
- pageNames[]: list of all pages
- pageNameSet
- pageCount
    
## Implemented Methods

#### __init__(self,folder_path):
Construct a WikidWiki class using the folder_path as string.

#### readFileSystem(self):
Populates the PageName attributes of WikidWiki class from the filesystem


### Single Page Functions:
#### doesPageContainStringLowMem(self,pageName,srchstr):
#### doesPageContainString(self,pageName,srchstr):
#### __extractWikiFileNamesFromPath(self,wikiFilePath):
#### __getFilePathFromPageName(self,pageName):


### Get Pages:
#### getAllPages(self):
Returns a list of Pages found in data folder.

#### getPagesByFunction(self,matchFunction,matchValue=""):
Passed a match function and a match value to filter pages
Returns list of pageNames that return true when processed by function matchFunction(matchValue,pagename)

#### getPagesBySearchStr(self,srchstr):
Returns list of pageNames that contain the srchstr in their content

#### getPagesBySearchStrLowMem(self,srchstr):
Returns list of pageNames that contain the srchstr in their content

#### getPagesSetUnion(self,pn1,pn2):
Returns the union of pageName lists pn1 and pn2

#### getPagesSetIntersection(self,pn1,pn2):
Returns the intersection of pageName lists pn1 and pn2

#### getPagesSetDifference(self,pn1,pn2):
Returns the difference of pageName lists pn1 and pn2

#### getPagesSetSymmetricDifference(self,pn1,pn2):
Returns the symmetric difference (those in one or the other, but not both) of pageName lists pn1 and pn2



In [32]:
import os,glob
import mmap

from WikidWorker import hasWiki

class WikidWiki(object):
    """class for working with a wikidpad wiki from the filesystem."""

    def __init__(self,folder_path):
        """Construct a WikidWiki class using the folder_path as string."""
        # raise error if folder path doesn't exist or contains no wiki:
        if not os.path.exists(folder_path):
            raise Exception("folder " + folder_path + " does not exist!")
        if not hasWiki(folder_path):
            raise Exception("folder " + folder_path + " does not contain a wiki!")

        #set attributes:
        self.rootfolder = folder_path
        self.datafolder = os.path.join(folder_path,'data')
        self.readFileSystem()
        
    def readFileSystem(self):
        """Populates the PageName attributes of WikidWiki class from the filesystem"""
        self.pageNames = self.getAllPages()
        self.pageNamesSet = set(self.pageNames)
        self.pageCount = len(self.pageNames)
    
    def __extractWikiFileNamesFromPath(self,wikiFilePath):
        filename = os.path.split(wikiFilePath)[1]
        return os.path.splitext(filename)[0]

    def __getFilePathFromPageName(self,pageName):
        filename = os.path.join(self.datafolder,"%s.wiki" % pageName)
        return filename    
    
    def getAllPages(self):
        """Returns a list of Pages found in data folder."""
        filePathList = glob.glob(self.datafolder +'/*.wiki')
        return [self.__extractWikiFileNamesFromPath(x) for x in filePathList]
    
    def getPagesByFunction(self,matchFunction,matchValue=""):
        """ Passed a match function and a match value to filter pages
            Returns list of pageNames that return true when processed by function matchFunction(matchValue,pagename)
        """
        self.pageNames = self.getAllPages()
        if matchValue:
            return [x for x in self.pageNames if matchFunction(matchValue,x)]
        else:
            return [x for x in self.pageNames if matchFunction(x)]            

    # def doesPageContainStringLowMem(self,pageName,srchstr):
    #     filepath = self.__getFilePathFromPageName(pageName)
        
    #     if os.stat(filepath).st_size > 0:
    #         f = open(filepath,encoding="utf8")
    #         s = mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ)
    #         return s.find(srchstr) != -1
    #     return False
    
    def doesPageContainString(self,pageName,srchstr):
        filepath = self.__getFilePathFromPageName(pageName)
        return srchstr in open(filepath,encoding="utf8").read()

    def getPagesBySearchStr(self,srchstr):
        """Returns list of pageNames that contain the srchstr in their content"""
        return [x for x in self.pageNames if self.doesPageContainString(x,srchstr)]  
    
    # def getPagesBySearchStrLowMem(self,srchstr):
    #     """Returns list of pageNames that contain the srchstr in their content"""
    #     return [x for x in self.pageNames if self.doesPageContainStringLowMem(x,srchstr)] 
    
    def getPagesSetUnion(self,pn1,pn2):
        """Returns the union of pageName lists pn1 and pn2"""
        ps1 = set(pn1)
        ps2 = set(pn2)
        return ps1 | ps2
    
    def getPagesSetIntersection(self,pn1,pn2):
        """Returns the intersection of pageName lists pn1 and pn2"""
        ps1 = set(pn1)
        ps2 = set(pn2)
        return ps1 & ps2

    def getPagesSetDifference(self,pn1,pn2):
        """Returns the difference of pageName lists pn1 and pn2"""
        ps1 = set(pn1)
        ps2 = set(pn2)
        return ps1 - ps2

    def getPagesSetSymmetricDifference(self,pn1,pn2):
        """Returns the symmetric difference (those in one or the other, but not both) of pageName lists pn1 and pn2"""
        ps1 = set(pn1)
        ps2 = set(pn2)
        return ps1 ^ ps2
    
    def getPagesByTag(self, tag):
        """Passed a single tag and returns all pages containing that tag."""
        return self.getPagesBySearchStr("[tag:%s]" % tag)

    def getPagesByTags(self, tags):
        """Passed a list of tags and returns all pages containing ALL those tags."""
        pages = self.pageNames
        for t in tags:
            pages = self.getPagesSetIntersection(pages,self.getPagesByTag(t))
        return pages

    def getPagesWithNoTag(self):
        """Gets all pages containing NO tag."""
        return self.getPagesSetDifference(self.pageNames,self.getPagesBySearchStr("[tag"))

    def getPagesByCategory(self, cat):
        """Passed a single category and returns all pages containing that category link."""
        return self.getPagesBySearchStr("Category%s" % cat.capitalize())

    def getPagesByCategories(self, cats):
        """Passed a list of categories and returns all pages containing ALL those category links."""
        pages = self.pageNames
        for c in cats:
            pages = self.getPagesSetIntersection(pages,self.getPagesByCategory(c))
        return pages
    
    def getPagesByCatOrTag(self, val):
        """Returns all pages with either tag or category = val"""
        return self.getPagesSetUnion(self.getPagesByTag(val),self.getPagesByCategory(val))
    

In [33]:
jwik = WikidWiki("D:/DEV/repos/WikidTools/WikidLibPy/TestWiki")
l = ["dev","python"]
val = "work"
# p1 = jwik.getPagesByTags(t)
# p1 = jwik.getPagesByCategories(t)
p1 = jwik.getPagesByCatOrTag(val)
print("count = %i" % len(p1))
print(p1)
p2 = jwik.getPagesByCategory(val)
print("count = %i" % len(p2))
p3 = jwik.getPagesByTag(val)
print("count = %i" % len(p3))
p4 = jwik.getPagesSetUnion(p2,p3)
print("count = %i" % len(p4))
print(p4)
p5 = jwik.getPagesSetSymmetricDifference(p2,p3)
print("count = %i" % len(p5))
print(p5)


count = 2
{'TestThree', 'TestOne'}
count = 1
count = 1
count = 2
{'TestThree', 'TestOne'}
count = 2
{'TestThree', 'TestOne'}


In [34]:
jwik = WikidWiki("D:/DEV/repos/WikidTools/WikidLibPy/TestWiki")
l = ["dev","python"]
val = "work"

#jwik.doesPageContainString("TestThree","Category")

#jwik.doesPageContainStringLowMem("TestThree","Category")

jwik.getPagesBySearchStr("CategoryWork")
#print(p1)

p2 = jwik.getPagesByCategory(val)
print($"{p2}  count = {p2}")
# p3 = jwik.getPagesByTag(val)
# print("count = %i" % len(p3))
# p4 = jwik.getPagesSetUnion(p2,p3)
# print("count = %i" % len(p4))
# print(p4)
# p5 = jwik.getPagesSetSymmetricDifference(p2,p3)
# print("count = %i" % len(p5))
# print(p5)


count = 1


In [25]:
 def doesPageContainStringLowMem(self,pageName,srchstr):
        filepath = self.__getFilePathFromPageName(pageName)
        
        if os.stat(filepath).st_size > 0:
            f = open(filepath,encoding="utf8")
            s = mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ)
            return s.find(srchstr) != -1
        return False# Automated Tests:



In [ ]:
#TESTS
from nose.tools import assert_equal
#from nose.tools import assert_items_equal
from nose.tools import eq_  

class WikidWikiClassTest(object):
    
    def test(self):      
        tc = WikidWiki('TestWiki')
        
        assert_equal(tc.pageNames,['TestOne', 'TestThree', 'TestTwo'])
        assert_equal(tc.pageCount,3)
        
        eq_(tc.getPagesByFunction(PageNameContainsStringOne),['TestOne'])
        eq_(tc.getPagesByFunction(PageNameContainsString,'Two'),['TestTwo'])
        eq_(tc.getPagesByFunction(PageNameContainsString,'Three'),['TestThree'])
        assert_equal(set(tc.getPagesByFunction(PageNameContainsString,'Test')),set(['TestOne','TestTwo','TestThree']))
        
        print ('class ' + str(tc) + ' - PASSED ALL TESTS')
                     
#run tests
def PageNameContainsStringOne(pageName):
    return 'One' in pageName

def PageNameContainsString(s,pageName):
    return s in pageName

t = WikidWikiClassTest()
t.test()


## PageNameSet Class

In [ ]:
class PageNameSet (object):
    
    def __init__(self,pageName):
        
        self.filePath
        
        
        

# Methods todo

## PerformFunctionOnPages(PageList, function)
